安裝必要套件

In [ ]:
!pip install tensorflow scikit-learn pandas matplotlib tqdm

掛載 Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 設定雲端硬碟目錄
import os
MODEL_DIR = '/content/drive/MyDrive/rul_diffusion_fd001'
os.makedirs(MODEL_DIR, exist_ok=True)

Mounted at /content/drive


In [ ]:
!pip install optuna optuna-integration[tfkeras] -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 9.0 MB/s eta 0:00:00


In [ ]:
!pip install python-docx -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow

In [ ]:
!pip install tensorflow scikit-learn matplotlib python-docx -q

In [ ]:
!pip install optuna

In [ ]:
!pip install tensorflow optuna

In [ ]:
!pip install tensorflow scikit-learn pandas matplotlib tqdm

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from tqdm import tqdm

下載並載入 C-MAPSS 資料集（FD001)

In [ ]:
# 上傳 kaggle.json（只需做一次）
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"billlin01","key":"c42c112425b5d48a2926ec740846bfc2"}'}

設定 Kaggle API 權限並下載資料

In [ ]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)
!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# 以 NASA C-MAPSS 資料為例
# 替換成你要的 Kaggle dataset slug（網址最後一段）
!kaggle datasets download -d behrad3d/nasa-cmaps
!unzip -o nasa-cmaps.zip -d cmapss_data

Dataset URL: https://www.kaggle.com/datasets/behrad3d/nasa-cmaps
License(s): CC0-1.0
  0% 0.00/12.3M [00:00<?, ?B/s]
100% 12.3M/12.3M [00:00<00:00, 1.11GB/s]
Archive:  nasa-cmaps.zip
  inflating: cmapss_data/CMaps/Damage Propagation Modeling.pdf  
  inflating: cmapss_data/CMaps/RUL_FD001.txt  
  inflating: cmapss_data/CMaps/RUL_FD002.txt  
  inflating: cmapss_data/CMaps/RUL_FD003.txt  
  inflating: cmapss_data/CMaps/RUL_FD004.txt  
  inflating: cmapss_data/CMaps/readme.txt  
  inflating: cmapss_data/CMaps/test_FD001.txt  
  inflating: cmapss_data/CMaps/test_FD002.txt  
  inflating: cmapss_data/CMaps/test_FD003.txt  
  inflating: cmapss_data/CMaps/test_FD004.txt  
  inflating: cmapss_data/CMaps/train_FD001.txt  
  inflating: cmapss_data/CMaps/train_FD002.txt  
  inflating: cmapss_data/CMaps/train_FD003.txt  
  inflating: cmapss_data/CMaps/train_FD004.txt  
  inflating: cmapss_data/CMaps/x.txt  


讀取資料夾路徑

In [ ]:
import pandas as pd

train = pd.read_csv('/content/cmapss_data/CMaps/train_FD001.txt', sep='\s+', header=None)
print(train.head())

   0   1       2       3      4       5       6        7        8      9   \
0   1   1 -0.0007 -0.0004  100.0  518.67  641.82  1589.70  1400.60  14.62   
1   1   2  0.0019 -0.0003  100.0  518.67  642.15  1591.82  1403.14  14.62   
2   1   3 -0.0043  0.0003  100.0  518.67  642.35  1587.99  1404.20  14.62   
3   1   4  0.0007  0.0000  100.0  518.67  642.35  1582.79  1401.87  14.62   
4   1   5 -0.0019 -0.0002  100.0  518.67  642.37  1582.85  1406.22  14.62   

   ...      16       17       18      19    20   21    22     23     24  \
0  ...  521.66  2388.02  8138.62  8.4195  0.03  392  2388  100.0  39.06   
1  ...  522.28  2388.07  8131.49  8.4318  0.03  392  2388  100.0  39.00   
2  ...  522.42  2388.03  8133.23  8.4178  0.03  390  2388  100.0  38.95   
3  ...  522.86  2388.08  8133.83  8.3682  0.03  392  2388  100.0  38.88   
4  ...  522.19  2388.04  8133.80  8.4294  0.03  393  2388  100.0  38.90   

        25  
0  23.4190  
1  23.4236  
2  23.3442  
3  23.3739  
4  23.4044  

[5 rows

資料預處理（選感測器、平滑、標準化並存scaler）

In [ ]:
import numpy as np
import pandas as pd
import os

MODEL_DIR = '/content/drive/MyDrive/rul_diffusion_fd004'
os.makedirs(MODEL_DIR, exist_ok=True)
max_rul = 130
columns = ['unit', 'time', 'op1', 'op2', 'op3'] + [f's{i}' for i in range(1, 22)]

# 分批處理
def process_and_save_unit(unit_id, df_unit, features):
    # 指數平滑
    for col in features:
        x = df_unit[col].copy()
        for i in range(1, len(x)):
            x.iloc[i] = 0.2 * x.iloc[i] + 0.8 * x.iloc[i-1]
        df_unit[col] = x
    # 儲存
    df_unit.to_csv(f"{MODEL_DIR}/unit_{unit_id}.csv", index=False, float_format="%.5f")

train = pd.read_csv('/content/cmapss_data/CMaps/train_FD004.txt', sep='\s+', header=None)
train.columns = columns
train['RUL_linear'] = train.groupby('unit')['time'].transform('max') - train['time']
train['RUL_linear'] = train['RUL_linear'].clip(upper=max_rul)

# 挑掉無變異sensor
sensor_cols = [f's{i}' for i in range(1, 22)]
sensor_cols_keep = [col for col in sensor_cols if train[col].std() >= 1e-3]

# 分段線性RUL標註（可參照你前面的 Algorithm1）
def generate_piecewise_rul_algorithm1(df, feature_cols, window_size=12, threshold=0.2, max_rul=130):
    results = []
    for unit_id, group in df.groupby('unit'):
        group = group.sort_values('time').reset_index(drop=True)
        sensor_data = group[feature_cols].values
        num_cycles = len(group)
        num_windows = num_cycles // window_size

        if num_windows < 3:
            irul = num_cycles
            group['iRUL'] = irul
            group['RUL_piecewise'] = np.clip(irul - group['time'], 0, max_rul)
            results.append((unit_id, group))
            continue

        centroids = [np.mean(sensor_data[i * window_size:(i + 1) * window_size], axis=0)
                     for i in range(num_windows)]
        base = centroids[0]
        degradation_found = False

        for i in range(2, num_windows):
            dist_sq = np.sum((centroids[i] - base) ** 2)
            if dist_sq >= threshold:
                degradation_start = i * window_size
                irul = num_cycles - degradation_start
                degradation_found = True
                break

        if not degradation_found:
            irul = num_cycles

        rul_piecewise = [
            irul if t <= (num_cycles - irul) else irul - (t - (num_cycles - irul))
            for t in range(num_cycles)
        ]
        group['iRUL'] = irul
        group['RUL_piecewise'] = np.clip(rul_piecewise, 0, max_rul)
        results.append((unit_id, group))
    return results

results = generate_piecewise_rul_algorithm1(train, sensor_cols_keep, window_size=12, threshold=0.2, max_rul=max_rul)

# 只做特徵選擇/感測器選取，然後直接分unit儲存
selected_sensors = sensor_cols_keep[:8]  # 預設先隨便選8個感測器
features = selected_sensors + ['op1', 'op2', 'op3']

for unit_id, df_unit in results:
    process_and_save_unit(unit_id, df_unit, features)
    # 釋放記憶體
    del df_unit

print('✅ 每個unit已個別分批儲存，絕不會爆RAM。')


✅ 每個unit已個別分批儲存，絕不會爆RAM。


In [ ]:
import pandas as pd
import numpy as np
import joblib
import glob

MODEL_DIR = '/content/drive/MyDrive/rul_diffusion_fd004'
feature_names = ['s2','s3','s4','s7','s11','s12','s15','s17','op1','op2','op3']  # 你選到的8+3特徵
X = []
for fname in glob.glob(f"{MODEL_DIR}/unit_*.csv"):
    df = pd.read_csv(fname)
    X.append(df[feature_names].values)
X = np.vstack(X).astype(np.float32)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
joblib.dump(scaler, f'{MODEL_DIR}/scaler_preprocessed.joblib')
joblib.dump(feature_names, f'{MODEL_DIR}/feature_names.pkl')
print('✅ 全域標準化fit完成')

# 再做一次transform
for fname in glob.glob(f"{MODEL_DIR}/unit_*.csv"):
    df = pd.read_csv(fname)
    df[feature_names] = scaler.transform(df[feature_names])
    df.to_csv(fname.replace('.csv', '_std.csv'), index=False, float_format="%.5f")
    os.remove(fname)  # 清掉原始
print('✅ 每個unit已標準化，且單次只佔用一個小unit檔，RAM零風險')


✅ 全域標準化fit完成


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but Sta

✅ 每個unit已標準化，且單次只佔用一個小unit檔，RAM零風險


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but Sta

產生 X_train 滑動視窗序列

In [ ]:
import numpy as np
import pandas as pd
import glob
import os

SEQ_LEN = 32
MODEL_DIR = '/content/drive/MyDrive/rul_diffusion_fd004'
feature_names = joblib.load(f'{MODEL_DIR}/feature_names.pkl')  # 讀進 ['s2',..., 'op3']
X_path_list, y_path_list, unit_path_list = [], [], []

X_batches, y_batches, unit_batches = [], [], []
batch_idx = 0
flush_every = 20  # 每20個unit flush一次（可依記憶體再縮小）

for count, fname in enumerate(sorted(glob.glob(f"{MODEL_DIR}/unit_*_std.csv"))):
    df = pd.read_csv(fname)
    arr = df[feature_names].values
    rul_piecewise = df['RUL_piecewise'].values
    unit = int(df['unit'].iloc[0])
    for i in range(len(arr) - SEQ_LEN + 1):
        X_batches.append(arr[i:i+SEQ_LEN])
        y_batches.append(rul_piecewise[i+SEQ_LEN-1])
        unit_batches.append(unit)
    # flush
    if (count+1) % flush_every == 0:
        np.save(f'{MODEL_DIR}/X_batch_{batch_idx}.npy', np.stack(X_batches).astype(np.float32))
        np.save(f'{MODEL_DIR}/y_batch_{batch_idx}.npy', np.array(y_batches).astype(np.float32))
        np.save(f'{MODEL_DIR}/unit_batch_{batch_idx}.npy', np.array(unit_batches).astype(int))
        X_path_list.append(f'{MODEL_DIR}/X_batch_{batch_idx}.npy')
        y_path_list.append(f'{MODEL_DIR}/y_batch_{batch_idx}.npy')
        unit_path_list.append(f'{MODEL_DIR}/unit_batch_{batch_idx}.npy')
        X_batches, y_batches, unit_batches = [], [], []
        print(f'Flush batch {batch_idx}')
        batch_idx += 1

# 處理最後一批
if X_batches:
    np.save(f'{MODEL_DIR}/X_batch_{batch_idx}.npy', np.stack(X_batches).astype(np.float32))
    np.save(f'{MODEL_DIR}/y_batch_{batch_idx}.npy', np.array(y_batches).astype(np.float32))
    np.save(f'{MODEL_DIR}/unit_batch_{batch_idx}.npy', np.array(unit_batches).astype(int))
    X_path_list.append(f'{MODEL_DIR}/X_batch_{batch_idx}.npy')
    y_path_list.append(f'{MODEL_DIR}/y_batch_{batch_idx}.npy')
    unit_path_list.append(f'{MODEL_DIR}/unit_batch_{batch_idx}.npy')
    print(f'Flush last batch {batch_idx}')

print("所有滑窗batch檔案已儲存，開始磁碟合併…")

# 合併所有 .npy 檔案
def concat_npy_batches(file_list):
    arrays = []
    for path in file_list:
        arrays.append(np.load(path, mmap_mode='r'))  # mmap省RAM
    return np.concatenate(arrays, axis=0)

X_all = concat_npy_batches(X_path_list)
y_all = concat_npy_batches(y_path_list)
unit_all = concat_npy_batches(unit_path_list)

print("滑動視窗完成:")
print("X_all shape:", X_all.shape)
print("y_all shape:", y_all.shape)

np.save(f'{MODEL_DIR}/X_all.npy', X_all)
np.save(f'{MODEL_DIR}/y_all.npy', y_all)
np.save(f'{MODEL_DIR}/unit_all.npy', unit_all)

# 刪除暫存小檔（可選）
for f in X_path_list + y_path_list + unit_path_list:
    os.remove(f)

print('✅ 全部滑窗序列合併並儲存完畢，不會爆RAM！')


Flush batch 0
Flush batch 1
Flush batch 2
Flush batch 3
Flush batch 4
Flush batch 5
Flush batch 6
Flush batch 7
Flush batch 8
Flush batch 9
Flush batch 10
Flush batch 11
Flush batch 12
Flush batch 13
Flush batch 14
Flush batch 15
Flush batch 16
Flush batch 17
Flush batch 18
Flush batch 19
Flush batch 20
Flush batch 21
Flush batch 22
Flush batch 23
Flush batch 24
Flush batch 25
Flush batch 26
Flush batch 27
Flush batch 28
Flush batch 29
Flush batch 30
Flush batch 31
Flush batch 32
Flush batch 33
Flush batch 34
Flush batch 35
Flush batch 36
Flush batch 37
Flush batch 38
Flush batch 39
Flush batch 40
Flush batch 41
Flush batch 42
Flush batch 43
Flush batch 44
Flush batch 45
Flush batch 46
Flush batch 47
Flush batch 48
Flush batch 49
Flush batch 50
Flush batch 51
Flush batch 52
Flush batch 53
Flush batch 54
Flush batch 55
Flush batch 56
Flush batch 57
Flush batch 58
Flush batch 59
Flush batch 60
Flush last batch 61
所有滑窗batch檔案已儲存，開始磁碟合併…
滑動視窗完成:
X_all shape: (261859, 32, 11)
y_all shape: (

Stage 1 Encoder自監督預訓練，儲存模型到雲端

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.optimizers import Adam
import os
import random

# ==== 重要！所有自訂Layer要加註冊 ====
@tf.keras.utils.register_keras_serializable()
class ReduceMean1D(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.reduce_mean(inputs, axis=-1, keepdims=True)

@tf.keras.utils.register_keras_serializable()
class ReduceMax1D(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.reduce_max(inputs, axis=-1, keepdims=True)

def cbam_block(inputs, ratio=8):
    channel = int(inputs.shape[-1])
    seq_len = int(inputs.shape[-2])
    avg_pool = layers.GlobalAveragePooling1D()(inputs)
    max_pool = layers.GlobalMaxPooling1D()(inputs)
    shared_dense_one = layers.Dense(channel // ratio, activation='relu')
    shared_dense_two = layers.Dense(channel)
    avg_fc = shared_dense_two(shared_dense_one(avg_pool))
    max_fc = shared_dense_two(shared_dense_one(max_pool))
    channel_att = layers.Add()([avg_fc, max_fc])
    channel_att = layers.Activation('sigmoid')(channel_att)
    channel_att = layers.Reshape((1, channel))(channel_att)
    x = layers.Multiply()([inputs, channel_att])
    avg_pool_sp = ReduceMean1D()(x)
    max_pool_sp = ReduceMax1D()(x)
    spatial = layers.Concatenate(axis=-1)([avg_pool_sp, max_pool_sp])
    spatial_att = layers.Conv1D(1, 7, padding='same', activation='sigmoid')(spatial)
    out = layers.Multiply()([x, spatial_att])
    return out

def resblock(x, filters, kernel=3, drop=0.1):
    h = layers.Conv1D(filters, kernel, padding='same')(x)
    h = layers.BatchNormalization()(h)
    h = layers.Activation('relu')(h)
    h = layers.Conv1D(filters, kernel, padding='same')(h)
    h = layers.BatchNormalization()(h)
    if x.shape[-1] != filters:
        x = layers.Conv1D(filters, 1, padding='same')(x)
    out = layers.Add()([x, h])
    out = layers.Activation('relu')(out)
    out = layers.Dropout(drop)(out)
    return out

# 超參數
SEQ_LEN = 32
D = 16
T = 1000
BATCH_SIZE = 128
EPOCHS_STAGE1 = 300
GLOBAL_SEED = 42

MODEL_DIR = '/content/drive/MyDrive/rul_diffusion_fd004'
os.makedirs(MODEL_DIR, exist_ok=True)
np.random.seed(GLOBAL_SEED)
tf.random.set_seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)

X_all = np.load(f'{MODEL_DIR}/X_all.npy')
N_SENSORS = X_all.shape[-1]

def cosine_beta_schedule(timesteps, s=0.008):
    steps = timesteps + 1
    x = np.linspace(0, timesteps, steps)
    f = np.cos(((x / timesteps) + s) / (1 + s) * np.pi / 2) ** 2
    alphas_cumprod = f / f[0]
    betas = np.clip(1 - (alphas_cumprod[1:] / alphas_cumprod[:-1]), 0, 0.999)
    return betas
betas = cosine_beta_schedule(T, s=0.008)
alphas = 1 - betas
alphas_cumprod = np.cumprod(alphas)

def get_timestep_embedding(timesteps, dim=D):
    timesteps = tf.convert_to_tensor(timesteps, dtype=tf.float32)
    timesteps = tf.reshape(timesteps, [-1, 1])
    half_dim = dim // 2
    emb = tf.math.log(10000.0) / (half_dim - 1)
    emb = tf.exp(tf.range(half_dim, dtype=tf.float32) * -emb)
    emb = timesteps * emb
    emb = tf.concat([tf.sin(emb), tf.cos(emb)], axis=-1)
    return emb

def diffusion_dataset(X, batch_size, T, alphas_cumprod, D):
    def generator():
        dataset_size = len(X)
        while True:
            idxs = np.random.permutation(dataset_size)
            for i in range(0, dataset_size, batch_size):
                batch_idx = idxs[i:i+batch_size]
                x_start = X[batch_idx]
                b = len(x_start)
                t = np.random.randint(1, T + 1, size=b)
                tau = get_timestep_embedding(t, D).numpy()
                noise = np.random.randn(*x_start.shape).astype(np.float32)
                sqrt_alpha = np.sqrt(alphas_cumprod[t - 1])[:, None, None]
                sqrt_one_minus_alpha = np.sqrt(1 - alphas_cumprod[t - 1])[:, None, None]
                x_t = sqrt_alpha * x_start + sqrt_one_minus_alpha * noise
                yield {"input_1": x_t, "input_2": tau}, noise
    output_signature = (
        {"input_1": tf.TensorSpec(shape=(None, SEQ_LEN, N_SENSORS), dtype=tf.float32),
         "input_2": tf.TensorSpec(shape=(None, D), dtype=tf.float32)},
        tf.TensorSpec(shape=(None, SEQ_LEN, N_SENSORS), dtype=tf.float32)
    )
    return tf.data.Dataset.from_generator(generator, output_signature=output_signature).prefetch(tf.data.AUTOTUNE)

def build_encoder_res(input_dim, embed_dim, seq_len):
    input_x = Input(shape=(seq_len, input_dim), name='input_1')
    input_tau = Input(shape=(embed_dim,), name='input_2')
    t_embed = layers.Dense(embed_dim, activation='swish')(input_tau)
    t_embed = layers.Dense(seq_len * input_dim, activation='swish')(t_embed)
    t_proj_exp = layers.Reshape((seq_len, input_dim))(t_embed)
    x = layers.Add()([input_x, t_proj_exp])
    x = resblock(x, 128, drop=0.1)
    x = resblock(x, 128, drop=0.1)
    x = cbam_block(x)
    x = resblock(x, 64, drop=0.1)
    output = layers.Conv1D(input_dim, 1, padding='same')(x)
    return Model([input_x, input_tau], output)

encoder_full = build_encoder_res(N_SENSORS, D, SEQ_LEN)
encoder_full.compile(optimizer=Adam(learning_rate=5e-4), loss='mse')

train_ds = diffusion_dataset(X_all, BATCH_SIZE, T, alphas_cumprod, D)
steps_per_epoch = len(X_all) // BATCH_SIZE

history = encoder_full.fit(
    train_ds,
    epochs=EPOCHS_STAGE1,
    steps_per_epoch=steps_per_epoch,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)]
)

embedding_layer_name = encoder_full.layers[-2].name  # Conv1D 輸出層之前
embedding_output = encoder_full.get_layer(embedding_layer_name).output
embedding_gap = layers.GlobalAveragePooling1D()(embedding_output)
encoder_embedding = Model(encoder_full.inputs, embedding_gap)

encoder_full.save(f'{MODEL_DIR}/stage1_encoder_full.keras')
encoder_embedding.save(f'{MODEL_DIR}/stage1_encoder_embedding.keras')
print("✅ Stage1 預訓練完成，模型與embedding已儲存。")


Epoch 1/300
2045/2045 ━━━━━━━━━━━━━━━━━━━━ 25s 5ms/step - loss: 0.3739
Epoch 2/300


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


2045/2045 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - loss: 0.1536
Epoch 3/300
2045/2045 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.1389
Epoch 4/300
2045/2045 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.1305
Epoch 5/300
2045/2045 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.1253
Epoch 6/300
2045/2045 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.1199
Epoch 7/300
2045/2045 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.1146
Epoch 8/300
2045/2045 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.1101
Epoch 9/300
2045/2045 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.1052
Epoch 10/300
2045/2045 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.1003
Epoch 11/300
2045/2045 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.0969
Epoch 12/300
2045/2045 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.0940
Epoch 13/300
2045/2045 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.0923
Epoch 14/300
2045/2045 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.0896
Epoch 15/300
2045/2045 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.0882
Epoch 16/300
20

Stage 2 聯合模型訓練與儲存

In [ ]:
!pip install -q optuna optuna-integration[tfkeras]

import os
import optuna
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.optimizers import AdamW
from optuna.integration import TFKerasPruningCallback
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
import random

# -- Stage1 custom layer定義要再複製一次 --
@tf.keras.utils.register_keras_serializable()
class ReduceMean1D(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.reduce_mean(inputs, axis=-1, keepdims=True)

@tf.keras.utils.register_keras_serializable()
class ReduceMax1D(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.reduce_max(inputs, axis=-1, keepdims=True)

def cbam_block(inputs, ratio=8):
    channel = int(inputs.shape[-1])
    seq_len = int(inputs.shape[-2])
    avg_pool = layers.GlobalAveragePooling1D()(inputs)
    max_pool = layers.GlobalMaxPooling1D()(inputs)
    shared_dense_one = layers.Dense(channel // ratio, activation='relu')
    shared_dense_two = layers.Dense(channel)
    avg_fc = shared_dense_two(shared_dense_one(avg_pool))
    max_fc = shared_dense_two(shared_dense_one(max_pool))
    channel_att = layers.Add()([avg_fc, max_fc])
    channel_att = layers.Activation('sigmoid')(channel_att)
    channel_att = layers.Reshape((1, channel))(channel_att)
    x = layers.Multiply()([inputs, channel_att])
    avg_pool_sp = ReduceMean1D()(x)
    max_pool_sp = ReduceMax1D()(x)
    spatial = layers.Concatenate(axis=-1)([avg_pool_sp, max_pool_sp])
    spatial_att = layers.Conv1D(1, 7, padding='same', activation='sigmoid')(spatial)
    out = layers.Multiply()([x, spatial_att])
    return out

# -- 其他全域參數 --
GLOBAL_SEED = 42
np.random.seed(GLOBAL_SEED)
tf.random.set_seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)

MODEL_DIR = '/content/drive/MyDrive/rul_diffusion_fd004'
SEQ_LEN, D, T = 32, 16, 1000

X_all = np.load(f'{MODEL_DIR}/X_all.npy')
y_all = np.load(f'{MODEL_DIR}/y_all.npy')
unit_all = np.load(f'{MODEL_DIR}/unit_all.npy')

gkf = GroupKFold(n_splits=5)
split = list(gkf.split(X_all, y_all, groups=unit_all))[0]
train_idx, val_idx = split
X_train, y_train = X_all[train_idx], y_all[train_idx]
X_val, y_val = X_all[val_idx], y_all[val_idx]

def get_timestep_embedding_np(timesteps, dim):
    timesteps = np.array(timesteps).reshape(-1, 1)
    half_dim = dim // 2
    emb = np.log(10000) / (half_dim - 1)
    emb = np.exp(np.arange(half_dim) * -emb)
    emb = timesteps * emb
    emb = np.concatenate([np.sin(emb), np.cos(emb)], axis=1)
    return emb.astype(np.float32)

t_train = np.random.randint(1, T + 1, size=len(X_train))
tau_train = get_timestep_embedding_np(t_train, D)
t_val = np.random.randint(1, T + 1, size=len(X_val))
tau_val = get_timestep_embedding_np(t_val, D)

# ==== 這裡必須傳 custom_objects，且把所有自訂層都列進來 ====
custom_objects = {
    'ReduceMean1D': ReduceMean1D,
    'ReduceMax1D': ReduceMax1D,
    'Functional': tf.keras.models.Model,   # 可省略，這是Keras內建
    # 若還有CBAM block自訂層也可加
}

encoder = tf.keras.models.load_model(
    f'{MODEL_DIR}/stage1_encoder_embedding.keras',
    compile=False,
    safe_mode=False,
    custom_objects=custom_objects
)
encoder.trainable = False  # 建議凍結

def build_rul_predictor(input_dim, hidden_units, dropout, activation):
    inputs = Input(shape=(input_dim,))
    x = inputs
    for h in hidden_units:
        x = layers.Dense(h, activation=activation)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(dropout)(x)
    output = layers.Dense(1)(x)
    return Model(inputs, output)

def custom_asymmetric_loss(lambda_high=0.05):
    def loss(y_true, y_pred):
        error = y_pred - y_true
        loss = tf.where(error < 0, 2 * tf.square(error), tf.square(error))
        base = tf.reduce_mean(loss)
        high_bias = tf.reduce_mean(tf.square(tf.nn.relu(y_pred - 110)))
        return base + lambda_high * high_bias
    return loss

def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 5e-4, log=True)
    dropout = trial.suggest_float("dropout", 0.08, 0.18)
    num_layers = trial.suggest_int("num_layers", 2, 3)
    hidden_units = [trial.suggest_int(f"n_units_l{i}", 128, 384, step=64) for i in range(num_layers)]
    batch_size = trial.suggest_categorical("batch_size", [32, 64])
    activation = trial.suggest_categorical("activation", ["relu", "swish"])
    lambda_high = trial.suggest_float("lambda_high", 0.01, 0.08)
    patience_es = trial.suggest_int("patience_es", 10, 14)
    patience_rlr = trial.suggest_int("patience_rlr", 3, 5)

    input_x = Input(shape=(SEQ_LEN, X_train.shape[-1]))
    input_t = Input(shape=(D,))
    encoded = encoder([input_x, input_t])
    mlp = build_rul_predictor(encoded.shape[-1], hidden_units, dropout, activation)
    output = mlp(encoded)
    joint_model = Model([input_x, input_t], output)
    joint_model.compile(optimizer=AdamW(learning_rate=learning_rate), loss=custom_asymmetric_loss(lambda_high))
    joint_model.fit([X_train, tau_train], y_train,
                    validation_data=([X_val, tau_val], y_val),
                    epochs=30, batch_size=batch_size, verbose=0,
                    callbacks=[
                        tf.keras.callbacks.EarlyStopping(patience=patience_es, restore_best_weights=True),
                        tf.keras.callbacks.ReduceLROnPlateau(patience=patience_rlr, factor=0.5, min_lr=1e-7),
                        TFKerasPruningCallback(trial, "val_loss")
                    ])
    y_pred = joint_model.predict([X_val, tau_val], batch_size=batch_size).flatten()
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    return rmse

study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=GLOBAL_SEED))
study.optimize(objective, n_trials=20, timeout=1800)

best = study.best_params
best_hidden = [best[f"n_units_l{i}"] for i in range(best["num_layers"])]

input_x = Input(shape=(SEQ_LEN, X_train.shape[-1]), name="stage2_input_x")
input_t = Input(shape=(D,), name="stage2_input_t")
encoded = encoder([input_x, input_t])
mlp = build_rul_predictor(encoded.shape[-1], best_hidden, best['dropout'], best['activation'])
output = mlp(encoded)
final_model = Model([input_x, input_t], output, name="fd004_joint_model")

final_model.compile(
    optimizer=AdamW(learning_rate=best['learning_rate']),
    loss=custom_asymmetric_loss(best['lambda_high'])
)
history = final_model.fit(
    [X_train, tau_train], y_train,
    validation_data=([X_val, tau_val], y_val),
    epochs=50,
    batch_size=best['batch_size'],
    verbose=2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=best['patience_es'], restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(patience=best['patience_rlr'], factor=0.5, min_lr=1e-7)
    ]
)
final_model.save(f'{MODEL_DIR}/stage2_joint_model_best.keras')
print("✅ Stage2 訓練與儲存完成！")

y_pred = final_model.predict([X_val, tau_val], batch_size=best['batch_size']).flatten()
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"✅ 最佳參數: {best}")
print(f"✅ 驗證集最小 RMSE (piecewise RUL): {rmse:.4f}")

import pandas as pd
df_trials = study.trials_dataframe()
df_trials.to_csv(f"{MODEL_DIR}/optuna_search_results.csv", index=False)
print(f"✅ Optuna搜尋結果已儲存：{MODEL_DIR}/optuna_search_results.csv")


[I 2025-06-06 15:57:18,289] A new study created in memory with name: no-name-cbae6ec3-55e8-403c-8f72-8aa05e1928bc


821/821 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step


[I 2025-06-06 16:01:21,039] Trial 0 finished with value: 42.52505723880995 and parameters: {'learning_rate': 0.00018272261776066238, 'dropout': 0.17507143064099162, 'num_layers': 3, 'n_units_l0': 256, 'n_units_l1': 128, 'n_units_l2': 128, 'batch_size': 64, 'activation': 'swish', 'lambda_high': 0.011440914600706172, 'patience_es': 14, 'patience_rlr': 5}. Best is trial 0 with value: 42.52505723880995.


1641/1641 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step


[I 2025-06-06 16:08:18,606] Trial 1 finished with value: 42.53833433146287 and parameters: {'learning_rate': 0.00014074036373847487, 'dropout': 0.09818249672071006, 'num_layers': 2, 'n_units_l0': 192, 'n_units_l1': 256, 'batch_size': 32, 'activation': 'relu', 'lambda_high': 0.03045012539746527, 'patience_es': 11, 'patience_rlr': 4}. Best is trial 0 with value: 42.52505723880995.


...

1641/1641 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step


[I 2025-06-06 16:26:44,793] Trial 3 finished with value: 42.71211714691599 and parameters: {'learning_rate': 0.0003007865147526456, 'dropout': 0.12401524937396013, 'num_layers': 2, 'n_units_l0': 256, 'n_units_l1': 128, 'batch_size': 32, 'activation': 'relu', 'lambda_high': 0.04640476148244676, 'patience_es': 12, 'patience_rlr': 3}. Best is trial 0 with value: 42.52505723880995.


821/821 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step


[I 2025-06-06 16:32:42,103] Trial 4 finished with value: 42.442576670757454 and parameters: {'learning_rate': 0.0004761135828626518, 'dropout': 0.15751328233611145, 'num_layers': 3, 'n_units_l0': 384, 'n_units_l1': 256, 'n_units_l2': 384, 'batch_size': 64, 'activation': 'swish', 'lambda_high': 0.03720741027826374, 'patience_es': 11, 'patience_rlr': 5}. Best is trial 4 with value: 42.442576670757454.


Epoch 1/50
3272/3272 - 25s - 8ms/step - loss: 4797.4868 - val_loss: 2138.3118 - learning_rate: 4.7611e-04
Epoch 2/50
3272/3272 - 11s - 3ms/step - loss: 2138.7810 - val_loss: 2308.9617 - learning_rate: 4.7611e-04
Epoch 3/50
3272/3272 - 11s - 3ms/step - loss: 2128.9133 - val_loss: 2119.1873 - learning_rate: 4.7611e-04
Epoch 4/50
3272/3272 - 11s - 3ms/step - loss: 2120.1880 - val_loss: 2184.6438 - learning_rate: 4.7611e-04
Epoch 5/50
3272/3272 - 11s - 3ms/step - loss: 2107.8728 - val_loss: 2115.4612 - learning_rate: 4.7611e-04
Epoch 6/50
3272/3272 - 11s - 3ms/step - loss: 2103.1162 - val_loss: 2103.7573 - learning_rate: 4.7611e-04
Epoch 7/50
3272/3272 - 11s - 3ms/step - loss: 2098.8103 - val_loss: 2152.5334 - learning_rate: 4.7611e-04
Epoch 8/50
3272/3272 - 11s - 3ms/step - loss: 2098.0364 - val_loss: 2113.7134 - learning_rate: 4.7611e-04
Epoch 9/50
3272/3272 - 11s - 3ms/step - loss: 2093.5242 - val_loss: 2104.2410 - learning_rate: 4.7611e-04
Epoch 10/50
3272/3272 - 11s - 3ms/step - loss:

Test資料集預處理

In [ ]:
import numpy as np
import pandas as pd
import joblib
import os

MODEL_DIR = '/content/drive/MyDrive/rul_diffusion_fd004'
SEQ_LEN = 32
D = 16  # embedding維度（須與訓練一致）
max_rul = 130

scaler = joblib.load(f'{MODEL_DIR}/scaler_preprocessed.joblib')
feature_names = joblib.load(f'{MODEL_DIR}/feature_names.pkl')
columns = ['unit', 'time', 'op1', 'op2', 'op3'] + [f's{i}' for i in range(1, 22)]

test = pd.read_csv('/content/cmapss_data/CMaps/test_FD004.txt', sep='\s+', header=None)
test.columns = columns
rul_truth = pd.read_csv('/content/cmapss_data/CMaps/RUL_FD004.txt', header=None, names=['RUL'])

# 1. 指數平滑（和訓練一致）
def exponential_smoothing(series, alpha=0.2):
    result = [series.iloc[0]]
    for n in range(1, len(series)):
        result.append(alpha * series.iloc[n] + (1 - alpha) * result[-1])
    return pd.Series(result, index=series.index)
for col in feature_names:
    test[col] = test.groupby('unit')[col].transform(lambda x: exponential_smoothing(x, alpha=0.2))

# 2. 標準化（用訓練集的 scaler）
test[feature_names] = scaler.transform(test[feature_names])

# 3. 產生滑動視窗（只取每unit最後SEQ_LEN，模擬RUL預測場景）
X_test_list, timestep_list, units_list = [], [], []
for unit in sorted(test['unit'].unique()):
    df_unit = test[test['unit'] == unit]
    arr = df_unit[feature_names].values
    if len(arr) >= SEQ_LEN:
        X_test_list.append(arr[-SEQ_LEN:])                 # 取最後SEQ_LEN點
        timestep_list.append(df_unit['time'].values[-1])   # 對應的最後時間
        units_list.append(unit)
X_test = np.stack(X_test_list).astype(np.float32)

# 4. timestep embedding（Diffusion/Transformer式時間特徵）
def get_timestep_embedding_np(timesteps, dim):
    timesteps = np.array(timesteps).reshape(-1, 1)
    half_dim = dim // 2
    emb = np.log(10000) / (half_dim - 1)
    emb = np.exp(np.arange(half_dim) * -emb)
    emb = timesteps * emb
    emb = np.concatenate([np.sin(emb), np.cos(emb)], axis=1)
    return emb.astype(np.float32)
tau_test = get_timestep_embedding_np(timestep_list, D)

# 5. 真實RUL對齊
unit_arr = np.array(units_list)
# rul_truth第i行對應unit=i+1（RUL順序無誤），但更保險還是對齊：
y_test = rul_truth.iloc[unit_arr - 1, 0].values

# 6. 儲存
np.save(f'{MODEL_DIR}/X_test.npy', X_test)
np.save(f'{MODEL_DIR}/tau_test.npy', tau_test)
np.save(f'{MODEL_DIR}/units_list.npy', unit_arr)
np.save(f'{MODEL_DIR}/y_test.npy', y_test)

print(f"✅ Test 視窗數: {len(X_test)}")
print(f"🧪 對齊 unit 數量: {len(np.unique(units_list))}")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


✅ Test 視窗數: 237
🧪 對齊 unit 數量: 237


Test驗證、計算RMSE與NASA Score、繪圖並存檔

In [ ]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE
from tensorflow.keras.models import load_model
import tensorflow as tf
import os

MODEL_DIR = '/content/drive/MyDrive/rul_diffusion_fd004'

# === 1. 載入 test set 滑窗資料與 RUL ground truth ===
X_test = np.load(f'{MODEL_DIR}/X_test.npy')
tau_test = np.load(f'{MODEL_DIR}/tau_test.npy')
units_list = np.load(f'{MODEL_DIR}/units_list.npy').astype(int)
y_true = np.load(f'{MODEL_DIR}/y_test.npy')

# === 2. 抓最佳 lambda_high（或直接寫死最佳值也可） ===
optuna_result_csv = f'{MODEL_DIR}/optuna_search_results.csv'
optuna_df = pd.read_csv(optuna_result_csv)
if 'params_lambda_high' in optuna_df.columns:
    best_trial = optuna_df.loc[optuna_df['value'].idxmin()]
    best_lambda_high = best_trial['params_lambda_high']
else:
    best_lambda_high = 0.1  # 預設值

def custom_rul_loss(lambda_high=best_lambda_high):
    def loss(y_true, y_pred):
        # 較推薦如下，跟你訓練的一致（clip是非必要的，純粹做模型輸出限制也可移除）
        base = tf.reduce_mean(tf.square(y_pred - y_true))
        high_bias = tf.reduce_mean(tf.square(tf.nn.relu(y_pred - 110)))
        return base + lambda_high * high_bias
    return loss

# === 3. 載入最佳 Stage2 聯合模型 ===
# 1️⃣ 請確定模型存檔格式正確（通常 ".keras" or ".h5"），建議與訓練保持一致！
# 2️⃣ **若有用自訂層，一定要補 custom_objects 內容！**
joint_model = load_model(
    f'{MODEL_DIR}/stage2_joint_model_best.keras',      # ← 跟訓練階段一致
    custom_objects={"loss": custom_rul_loss()}
)

# === 4. 預測 ===
y_pred = joint_model.predict([X_test, tau_test], verbose=1).flatten()
y_pred = np.clip(y_pred, 0, 130)

# === 5. NASA Score 計算 ===
def nasa_score(y_true, y_pred):
    score = 0
    for true, pred in zip(y_true, y_pred):
        d = pred - true
        if d < 0:
            score += np.exp(-d / 13) - 1
        else:
            score += np.exp(d / 10) - 1
    return score

# === 6. 評分 ===
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
score = nasa_score(y_true, y_pred)
print(f'✅ Test RMSE: {rmse:.4f}')
print(f'✅ Test NASA Score: {score:.4f}')

# === 7. 儲存預測結果 ===
df_out = pd.DataFrame({
    'unit': units_list,
    'True_RUL': y_true,
    'Pred_RUL': y_pred,
    'Error': y_pred - y_true
})
df_out.to_csv(f'{MODEL_DIR}/test_pred_result.csv', index=False)
print(f"📁 預測結果已儲存：{MODEL_DIR}/test_pred_result.csv")

# === 8. 圖表: 預測 vs 真實 RUL ===
plt.figure(figsize=(10, 6))
plt.plot(y_true, label='True RUL', marker='o', alpha=0.8)
plt.plot(y_pred, label='Predicted RUL', marker='x', alpha=0.8)
plt.xlabel('Test Unit (Engine)')
plt.ylabel('RUL')
plt.legend()
plt.title(f'Predicted vs. True RUL (Test)\nRMSE={rmse:.2f} | NASA Score={score:.2f}')
plt.grid()
plt.tight_layout()
plt.savefig(f'{MODEL_DIR}/test_pred_vs_true_rul.png')
plt.close()

# === 9. 殘差分布圖 (Error vs True RUL) ===
plt.figure(figsize=(8,6))
plt.scatter(y_true, y_pred - y_true, alpha=0.75, c='royalblue', edgecolor='k')
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('True RUL')
plt.ylabel('Prediction Error (Predicted - True)')
plt.title('Residual Distribution (Error vs True RUL)')
plt.grid()
plt.tight_layout()
plt.savefig(f'{MODEL_DIR}/test_residual_scatter.png')
plt.close()

# === 10. 預測誤差直方圖 ===
plt.figure(figsize=(8,4))
plt.hist(y_pred - y_true, bins=25, color='skyblue', edgecolor='black')
plt.title('Prediction Error Histogram')
plt.xlabel('Prediction Error (Predicted - True)')
plt.ylabel('Count')
plt.grid()
plt.tight_layout()
plt.savefig(f'{MODEL_DIR}/test_error_histogram.png')
plt.close()

# === 11. t-SNE 可視化 Stage1 Encoder Embedding 特徵（**需 custom_objects**） ===
try:
    # -- 要傳入和訓練時完全一致的自訂層 (若有) --
    @tf.keras.utils.register_keras_serializable()
    class ReduceMean1D(tf.keras.layers.Layer):
        def call(self, inputs):
            return tf.reduce_mean(inputs, axis=-1, keepdims=True)
    @tf.keras.utils.register_keras_serializable()
    class ReduceMax1D(tf.keras.layers.Layer):
        def call(self, inputs):
            return tf.reduce_max(inputs, axis=-1, keepdims=True)

    encoder = load_model(
        f'{MODEL_DIR}/stage1_encoder_embedding.keras',
        compile=False,
        custom_objects={'ReduceMean1D': ReduceMean1D, 'ReduceMax1D': ReduceMax1D}
    )
    features = encoder.predict([X_test, tau_test])
    features_2d = TSNE(n_components=2, random_state=42).fit_transform(features)
    plt.figure(figsize=(7,6))
    sc = plt.scatter(features_2d[:,0], features_2d[:,1], c=y_true, cmap='viridis', s=20)
    plt.colorbar(sc, label='True RUL')
    plt.title("t-SNE of Encoder Embedding Features (Test set)")
    plt.xlabel("t-SNE Dim 1")
    plt.ylabel("t-SNE Dim 2")
    plt.savefig(f'{MODEL_DIR}/test_encoder_tsne.png')
    plt.close()
except Exception as e:
    print(f"[t-SNE 可視化失敗] {e}")

print(f"📈 圖片已儲存：\n{MODEL_DIR}/test_pred_vs_true_rul.png\n{MODEL_DIR}/test_residual_scatter.png\n{MODEL_DIR}/test_error_histogram.png\n{MODEL_DIR}/test_encoder_tsne.png")


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 313ms/step
✅ Test RMSE: 53.6204
✅ Test NASA Score: 539271.2027
📁 預測結果已儲存：/content/drive/MyDrive/rul_diffusion_fd004/test_pred_result.csv


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 137ms/step
📈 圖片已儲存：
/content/drive/MyDrive/rul_diffusion_fd004/test_pred_vs_true_rul.png
/content/drive/MyDrive/rul_diffusion_fd004/test_residual_scatter.png
/content/drive/MyDrive/rul_diffusion_fd004/test_error_histogram.png
/content/drive/MyDrive/rul_diffusion_fd004/test_encoder_tsne.png
